In [1]:
# default_exp matching

# Matching

> Functions related to matching

This notebook contains all functions related to matching.

Current ToDo here:

- Move the current Code to here

In [2]:
#hide
from nbdev.showdoc import *

# TBD

In [3]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_main.ipynb.
Converted 01_chem.ipynb.
Converted 02_io.ipynb.
Converted 03_fasta.ipynb.
Converted 04_feature_finding.ipynb.
Converted 05_search.ipynb.
Converted 06_score.ipynb.
Converted 07_recalibration.ipynb.
Converted 08_quantification.ipynb.
Converted 09_matching.ipynb.
Converted 10_constants.ipynb.
Converted 11_runner.ipynb.
Converted 12_settings.ipynb.
Converted 13_settings_template.ipynb.
Converted WebTest.ipynb.
Converted index.ipynb.
